# "I Fought the Law"
### Contesting Charges in Virginia's District Courts

In [159]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix

import ipywidgets as widgets
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

import plotly.plotly as py
import plotly.tools as pytools
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.offline as pyo
import cufflinks as cf

from IPython.display import HTML

import warnings
warnings.filterwarnings('ignore')

pytools.set_credentials_file(username='katerdowdy', api_key='hBCWsR3iY9a1feRSpU2A')

%matplotlib inline

np.random.seed(42)

In [5]:
df_summary = pd.read_csv('./summary_county_data.csv')
df_agg = pd.read_csv('./aggregate_charge_data.csv')
df_full = pd.read_csv('./2017_full.csv')

df_summary = df_summary.sort_values(by = 'Court')
counties = list(df_summary['Court'].unique())
df_agg = df_agg.sort_values(by = 'ChargeType')
charges = list(df_agg['ChargeType'].unique())

In [43]:
def chloropleth(selection):
    try:
        fips = df_summary['full_fips']
        values = df_summary[selection]
        step1 = values.mean() / 5
        step2 = (max(values) - values.mean()) / 5
        step3 = (max(values) / 10)
        num_endpoints = [np.round((values.mean() - (step1 * 4)), -3),
                    np.round((values.mean() - (step1 * 3)), -3),
                    np.round((values.mean() - (step1 * 2)), -3),
                    np.round((values.mean() - step1), -3),
                    np.round((values.mean()), -3),
                    np.round((max(values) - (step2 * 4)), -3),
                    np.round((max(values) - (step2 * 3)), -3),
                    np.round((max(values) - (step2 * 2)), -3),
                    np.round((max(values) - step2), -3)]
        prop_endpoints = [.1, .2, .3, .4, .5, .6, .8, 1, 1.2]
        even_endpoints = [step3, (step3 * 2), (step3 * 3), (step3 * 4), (step3 * 5),
                         (step3 * 6), (step3 * 7), (step3 * 8), (step3 * 9)]
    
        colorscale = ["#eafcfd", "#b7e0e4", "#85c5d3", "#60a7c7", "#4989bc",
               "#3e6ab0", "#3d4b94", "#323268", "#1d1d3b", "#030512"]
        
        fig = ff.create_choropleth(fips = fips, 
                           values = values,
                          scope = ['VA'],
                          county_outline={'color': 'rgb(169,169,169)', 'width': 1},
                           exponent_format=True,
                           #binning_endpoints = hearing_endpoints,
                           binning_endpoints = num_endpoints,
                          colorscale = colorscale,
                           legend_title=selection)
        return py.iplot(fig, filename=selection)
            
    except:
        fig = ff.create_choropleth(fips = fips, 
                           values = values,
                          scope = ['VA'],
                          county_outline={'color': 'rgb(169,169,169)', 'width': 1},
                           exponent_format=True,
                           #binning_endpoints = hearing_endpoints,
                           binning_endpoints = even_endpoints,
                          colorscale = colorscale,
                           legend_title=selection)
        return py.iplot(fig, filename=selection)

In [48]:
options = ['county_hearings', 'county_fines_charged', 'county_sentencing',
          'county_probation', 'defense_win_rate', 'Population',
          'fines_per_capita', 'hearings_per_capita']

#### Hearings by County

In [49]:
chloropleth('county_hearings')

#### Where Defenses Are Won

In [50]:
chloropleth('defense_win_rate')

In [46]:
chloro = interactive(chloropleth, selection = ['county_hearings', 'county_fines_charged', 
                                               'county_sentencing',
          'county_probation', 'defense_win_rate', 'Population',
          'fines_per_capita', 'hearings_per_capita'])

In [58]:
df_agg.columns

Index(['Unnamed: 0', 'CaseType', 'ChargeType', 'agg_charge',
       'agg_charge_overturn_rate', 'agg_contested_rate'],
      dtype='object')

In [96]:
def plot_agg_charges(casetype):
    trace1 = go.Bar(
        x=df_agg[df_agg['CaseType'] == casetype]['ChargeType'],
        y=df_agg[df_agg['CaseType'] == casetype]['agg_charge'],
        name='All Charges'
    )
    trace2 = go.Bar(
        x=df_agg[df_agg['CaseType'] == casetype]['ChargeType'],
        y=df_agg[df_agg['CaseType'] == casetype]['agg_charge'] * df_agg[df_agg['CaseType'] == casetype]['agg_contested_rate'],
        name='Defendants Went to Court'
    )

    trace3 = go.Bar(
        x=df_agg[df_agg['CaseType'] == casetype]['ChargeType'],
        y=df_agg[df_agg['CaseType'] == casetype]['agg_charge'] * df_agg[df_agg['CaseType'] == casetype]['agg_charge_overturn_rate'],
        name='Charges Dismissed/Overturned/Amended'
    )

    data = [trace1, trace2, trace3]
    
    layout = go.Layout(
    autosize=False,
    width=800,
    height=800,
    barmode='group',
    xaxis=dict(
        title='CHARGES',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='lightgrey'
        ),
        showticklabels=True,
        automargin=True,
        tickangle=45,
        tickfont=dict(
            family='Arial, sans-serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
)

    fig = go.Figure(data=data, layout=layout)
    return py.iplot(fig, filename='grouped-bar')

### Infractions

In [97]:
plot_agg_charges(casetype = 'Infraction')

### Misdemeanors

In [98]:
plot_agg_charges(casetype = 'Misdemeanor')

### Felonies

In [99]:
plot_agg_charges(casetype = 'Felony')

### Civil Violations

In [100]:
plot_agg_charges(casetype = 'Civil Violation')

### Modeling Outcomes

In [102]:
# drop features # complainant
drop_features = ['Unnamed: 0',
                 'level_0',
                             'index', 
                             'HearingDate', 
                             'HearingResult', 
                             #'HearingPlea',
                             'HearingContinuanceCode',
                             'HearingType',
                             'HearingCourtroom',
                             'fips',
                             'FiledDate',
                             'Locality',
                             'Status',  
                             'Address',
                             'Gender',
                             'Race',
                             'Charge', 
                             'CodeSection', 
                             'Contested',
                             'CaseType', 
                             'Class',
                             'OffenseDate', 
                             'ArrestDate', 
                             'AmendedCharge',
                             'AmendedCode', 
                             'AmendedCaseType', 
                             'FinalDisposition',
                             'ProbationTime', 
                             'ProbationStarts',
                             'SentenceTime', 
                             'SentenceSuspendedTime', 
                             'ProbationType',
                             'OperatorLicenseSuspensionTime',
                               'RestrictionEffectiveDate', 
                             'RestrictionEndDate',
                               'OperatorLicenseRestrictionCodes', 
                             'Fine', 
                             'Costs', 
                             'FineCostsDue',
                               'FineCostsPaid', 
                             'FineCostsPaidDate', 
                             'VASAP', 
                             'FineCostsPastDue',
                             'person_id', 
                             'person_id_freq',
                             'full_fips',
                             'Outcome_Positive', 
                             'Amended', 
                             'Total_Positive',
                            'ChargeType',
                            'Court']

In [103]:
df_full['DefenseAttorney'].fillna(0, inplace = True)
df_full['Complainant'].fillna(0, inplace = True)
df_full['HearingPlea'].fillna(0, inplace = True)

def log_odds(x):
    return np.exp(x)

In [114]:
def detail_logreg(county, charge):
    try:
        logreg = LogisticRegression()
        df = df_full[(df_full['Court'] == county) & (df_full['ChargeType'] == charge)]
        df_dummied = pd.get_dummies(df, columns = ['DefenseAttorney', 'Complainant', 'HearingPlea'], drop_first = True)
        df_model = df_dummied[df_dummied['Contested'] == 1]
        X = df_model.drop(columns = drop_features)
        features = X.columns
        y = df_model['Total_Positive']
        baseline = y.value_counts(normalize = True)
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)
        ss = StandardScaler()
        X_train_sc = ss.fit_transform(X_train)
        X_test_sc = ss.transform(X_test)
        logreg.fit(X_train_sc, y_train)
        cv_train = cross_val_score(logreg, X_train_sc, y_train)
        cv_test = cross_val_score(logreg, X_test_sc, y_test)
    
        coefficients = logreg.coef_
        coef_df = pd.DataFrame(coefficients, columns = features).T
        coef_df['change_odds_ratio'] = coef_df.apply(lambda x: log_odds(x))
        coef_df.rename(columns = {0: 'logreg_coefficient'}, inplace = True)
        coef_df_top = coef_df.sort_values(by = 'logreg_coefficient', ascending = False).head()
        coef_df_bottom = coef_df.sort_values(by = 'logreg_coefficient', ascending = False).tail()
        coef_df_all = pd.concat([coef_df_top, coef_df_bottom])
    
        print("Model for {} in {}".format(charge, county))
        print("Baseline:")
        print(baseline)
        print("-----")
        print("How Good is This Model?")
        print("Train Accuracy Scores:", cv_train, "Train Average Accuracy:", cv_train.mean())
        print("Test Accuracy Scores:", cv_test, "Test Average Accuracy:", cv_test.mean())
        print("-----")
        print("Factors that Help the Case")
        print(coef_df_top)
        print("-----")
        print("Factors that Hurt the Case")
        print(coef_df_bottom)
    except:
        pass

In [115]:
a = interactive(detail_logreg, county = counties, charge = charges)
display(a)

interactive(children=(Dropdown(description='county', options=('Accomack County', 'Albemarle County', 'Alexandr…

### Does Race Make a Difference?

In [142]:
def anova_race1(county, charge, *args):
    df_test = df_full[(df_full['ChargeType'] == charge) &
                         (df_full['Court'] == county) &
                         (df_full['Contested'] == 1)]
    if len(args[0]) == 2:
        test = stats.f_oneway(df_test[df_test['Race'] == str(args[0][0])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][1])]['Total_Positive'])
    elif len(args[0]) == 3:
        test = stats.f_oneway(df_test[df_test['Race'] == str(args[0][0])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][1])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][2])]['Total_Positive'])
    elif len(args[0]) == 4:
        test = stats.f_oneway(df_test[df_test['Race'] == str(args[0][0])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][1])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][2])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][3])]['Total_Positive'])
    elif len(args[0]) == 5:
        test = stats.f_oneway(df_test[df_test['Race'] == str(args[0][0])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][1])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][2])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][3])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][4])]['Total_Positive'])
    elif len(args[0]) == 6:
        test = stats.f_oneway(df_test[df_test['Race'] == str(args[0][0])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][1])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][2])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][3])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][4])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][5])]['Total_Positive'])

    percent_race = df_test['Race'].value_counts(normalize = True)
    p_value = test.pvalue
   
    print("Comparing mean outcomes for these populations contesting {} charges in {}:".format(charge, county))
    for i in args[0]:
        print("    ", i)
    print("P-value:", p_value)
    if p_value <= 0.01:
        print("The p-value is sufficiently small that we can reject the null hypothesis and accept the alternative hypothesis: that there is a statistically significant difference in defense outcomes for these groups based on race.")
    if p_value > 0.01:
        print("The p-value is not small enough to reject the null hypothesis. We cannot draw a conclusion about how outcomes differ by race for this charge.")
    print("--------------")
    print("Demographic makeup of defendees contesting {} charges in {}:".format(charge, county))
    print(percent_race)

In [157]:
def anova_race(county, charge, comparison):
    df_test = df_full[(df_full['ChargeType'] == charge) &
                         (df_full['Court'] == county) &
                         (df_full['Contested'] == 1)]
    if comparison == 'White, Black':
        test = stats.f_oneway(df_test[df_test['Race'] == 'White Caucasian(Non-Hispanic)']['Total_Positive'],
                          df_test[df_test['Race'] == 'Black(Non-Hispanic)']['Total_Positive'])
    elif comparison == 'White, Black, Latino':
        test = stats.f_oneway(df_test[df_test['Race'] == 'White Caucasian(Non-Hispanic)']['Total_Positive'],
                          df_test[df_test['Race'] == 'Black(Non-Hispanic)']['Total_Positive'],
                          df_test[df_test['Race'] == 'Hispanic']['Total_Positive'])
    elif comparison == 'White, Black, Latino, Asian Or Pacific Islander':
        test = stats.f_oneway(df_test[df_test['Race'] == 'White Caucasian(Non-Hispanic)']['Total_Positive'],
                          df_test[df_test['Race'] == 'Black(Non-Hispanic)']['Total_Positive'],
                          df_test[df_test['Race'] == 'Hispanic']['Total_Positive'],
                          df_test[df_test['Race'] == 'Asian Or Pacific Islander']['Total_Positive'])
    elif comparison == 'White, Black, Latino, Asian Or Pacific Islander, Native American':
        test = stats.f_oneway(df_test[df_test['Race'] == 'White Caucasian(Non-Hispanic)']['Total_Positive'],
                          df_test[df_test['Race'] == 'Black(Non-Hispanic)']['Total_Positive'],
                          df_test[df_test['Race'] == 'Hispanic']['Total_Positive'],
                          df_test[df_test['Race'] == 'Asian Or Pacific Islander']['Total_Positive'],
                          df_test[df_test['Race'] == 'American Indian']['Total_Positive'])

    percent_race = df_test['Race'].value_counts(normalize = True)
    p_value = test.pvalue
   
    print("Comparing mean outcomes for these populations contesting {} charges in {}:".format(charge, county))
    print(comparison)
    print("-----")
    print("P-value:", p_value)
    if p_value <= 0.01:
        print("YES. Race makes a difference.")
        print("The p-value is sufficiently small that we can reject the null hypothesis and accept the alternative hypothesis: "
              "that there is a statistically significant difference in defense outcomes for these groups based on race.")
    if p_value > 0.01:
        print("Inconclusive.")
        print("The p-value is not small enough to reject the null hypothesis. "
              "We cannot draw a conclusion about how outcomes differ by race for this charge.")
    print("--------------")
    print("Demographic makeup of defendants contesting {} charges in {}:".format(charge, county))
    print(percent_race)

In [145]:
comps = ['White, Black', 
        'White, Black, Latino',
        'White, Black, Latino, Asian Or Pacific Islander',
        'White, Black, Latino, Asian Or Pacific Islander, Native American']

In [158]:
anova_race = interactive(anova_race, county = counties, charge = charges, comparison = comps)
display(anova_race)

interactive(children=(Dropdown(description='county', options=('Accomack County', 'Albemarle County', 'Alexandr…

### Does Type of Defense Matter?

In [ ]:
# dummying variables
df_dummied = pd.get_dummies(df_full, columns = ['Court', 'Gender', 'Race',
                                                'ChargeType'], drop_first = True)

# slicing dataframe to just the records contested
df_model = df_dummied[df_dummied['Contested'] == 1]

In [ ]:
# setting X and y variables
y = df_model['Total_Positive']

X = df_model.drop(columns = ['Unnamed: 0',
                             'index', 
                             'HearingDate', 
                             'HearingResult', 
                             'HearingPlea',
                             'HearingType', 
                             'FiledDate', 
                             'Status', 
                             'DefenseAttorney', 
                             'Address',
                             'Charge', 
                             'CodeSection', 
                             'Contested',
                             'CaseType', 
                             'Class',
                             'OffenseDate', 
                             'ArrestDate', 
                             'AmendedCharge',
                             'AmendedCode', 
                             'AmendedCaseType', 
                             'FinalDisposition',
                             'Fine', 
                             'Costs', 
                             'person_id', 
                             'Outcome_Positive', 
                             'Amended', 
                             'Total_Positive',
                             'TimeSinceOffense',
                             'fips'])

In [ ]:
# checking to make sure no nulls
X.isnull().sum().head()

In [ ]:
# setting list of features
features = X.columns
features

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [ ]:
# checking baseline (59% positive outcome)
y_train.value_counts(normalize = True)

In [ ]:
# same for test
y_test.value_counts(normalize = True)

In [ ]:
# standard scale
ss = StandardScaler()

X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

### Logistic Regression

In [ ]:
logreg = LogisticRegression(penalty = 'l2')

In [ ]:
logreg.fit(X_train_sc, y_train)

In [ ]:
cv_scores = cross_val_score(logreg, X_train_sc, y_train)
print("Train CV Scores:", cv_scores)
print("Average Train CV Score:", cv_scores.mean())

In [ ]:
cv_scores = cross_val_score(logreg, X_test_sc, y_test)
print("Test CV Scores:", cv_scores)
print("Average Test CV Score:", cv_scores.mean())

Our train and test scores are pretty close so I don't think there's overfitting here. Accuracy is about 5% over baseline, which isn't great but also not terrible.

In [ ]:
coefficients = logreg.coef_

coef_df = pd.DataFrame(coefficients, columns = features).T

def log_odds(x):
    return np.exp(x)

coef_df['change_odds_ratio'] = coef_df.apply(lambda x: log_odds(x))

coef_df.rename(columns = {0: 'logreg_coefficient'}, inplace = True)

coef_df.sort_values(by = 'logreg_coefficient').head()

According to the Logistic Regression, race (Black, White, and Latino) and being charged with a Misdemeanor DWI/DUI or Seatbelt Infraction are the most negative coefficients.

In [ ]:
coef_df.sort_values(by = 'logreg_coefficient').tail()

According to the Logistic Regression, having a lawyer is the most positive coefficient, followed by being charged with speeding, license/permit/tag issues, and expired registration/inspection. 

##### Evaluating Confusion Matrix

In [ ]:
prob_preds = logreg.predict_proba(X_test_sc)
prob_preds

In [ ]:
plt.hist(prob_preds, bins = 20);

In [ ]:
outcome_preds = logreg.predict(X_test_sc)

In [ ]:
confusion_matrix(y_test, outcome_preds)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, outcome_preds).ravel()
print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)
print("-----")
print("Accuracy: %s" % ((tp + tn) / (tn + fp + fn + tp)))
print("Misclassification Rate: %s" % ((fp + fn) / (tn + fp + fn + tp)))
print("-----")
print("Sensitivity/Recall (True Positive Rate): %s" % ((tp) / (tp + fn)))
print("Specificity (True Negative Rate): %s" % ((tn) / (tn + fp)))
print("False Positive Rate: %s" % ((fp) / (tp + fn)))
print("Precision: %s" % ((tp) / (tp + fp)))

If we were to use a model to make recommendations to people who didn't have to contest their charges on whether or not to go to court (and risk having to pay court fees on top of their fines and a lawyer), we would want to be minimizing our False Positives (so get our Precision rate closer to 1.)

## Decision Tree

In [ ]:
dt = DecisionTreeClassifier()

In [ ]:
dt.fit(X_train_sc, y_train)

In [ ]:
# training scores
cv_scores = cross_val_score(dt, X_train_sc, y_train)
print("Train CV Scores:", cv_scores)
print("Average Train CV Score:", cv_scores.mean())

In [ ]:
# test scores
cv_scores = cross_val_score(logreg, X_test_sc, y_test)
print("Test CV Scores:", cv_scores)
print("Average Test CV Score:", cv_scores.mean())

The Decision Tree doesn't seem to be doing better than the Logistic Regression; I think the existing model is too simple.

In [ ]:
dt.n_features_

In [ ]:
importances = dt.feature_importances_
importance_dict = dict(zip(features, importances))

In [ ]:
importance_dict

The features that had the greatest feature importance weight: 
- 'Expired Registration/Inspection': 0.0248
- 'Gender_Male': 0.0237
- 'ChargeType_INF: Speeding': 0.0168
- 'Race_Black(Non-Hispanic)': 0.0129
- 'Race_White Caucasian(Non-Hispanic)': 0.0124
- 'Court_Fairfax County': 0.0123
- 'ChargeType_INF: Seatbelt': 0.0116

## Random Forest

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf.fit(X_train_sc, y_train)

In [ ]:
# training scores
cv_scores = cross_val_score(rf, X_train_sc, y_train)
print("Train CV Scores:", cv_scores)
print("Average Train CV Score:", cv_scores.mean())

In [ ]:
# test scores
cv_scores = cross_val_score(logreg, X_test_sc, y_test)
print("Test CV Scores:", cv_scores)
print("Average Test CV Score:", cv_scores.mean())

## County-level / Infraction-level Recommendation Models

In [ ]:
logreg = LogisticRegression()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()

In [ ]:
df_full.columns

In [ ]:
# drop features # complainant
drop_features = ['Unnamed: 0',
                 'level_0',
                             'index', 
                             'HearingDate', 
                             'HearingResult', 
                             #'HearingPlea',
                             'HearingContinuanceCode',
                             'HearingType',
                             'HearingCourtroom',
                             'fips',
                             'FiledDate',
                             'Locality',
                             'Status',  
                             'Address',
                             'Gender',
                             'Race',
                             'Charge', 
                             'CodeSection', 
                             'Contested',
                             'CaseType', 
                             'Class',
                             'OffenseDate', 
                             'ArrestDate', 
                             'AmendedCharge',
                             'AmendedCode', 
                             'AmendedCaseType', 
                             'FinalDisposition',
                             'ProbationTime', 
                             'ProbationStarts',
                             'SentenceTime', 
                             'SentenceSuspendedTime', 
                             'ProbationType',
                             'OperatorLicenseSuspensionTime',
                               'RestrictionEffectiveDate', 
                             'RestrictionEndDate',
                               'OperatorLicenseRestrictionCodes', 
                             'Fine', 
                             'Costs', 
                             'FineCostsDue',
                               'FineCostsPaid', 
                             'FineCostsPaidDate', 
                             'VASAP', 
                             'FineCostsPastDue',
                             'person_id', 
                             'person_id_freq',
                             'full_fips',
                             'Outcome_Positive', 
                             'Amended', 
                             'Total_Positive',
                            'ChargeType',
                            'Court']

In [ ]:
df_full['DefenseAttorney'].fillna(0, inplace = True)
df_full['Complainant'].fillna(0, inplace = True)
df_full['HearingPlea'].fillna(0, inplace = True)

In [ ]:
def log_odds(x):
    return np.exp(x)

In [ ]:
def detail_logreg(county, charge):
    try:
        logreg = LogisticRegression()
        df = df_full[(df_full['Court'] == county) & (df_full['ChargeType'] == charge)]
        df_dummied = pd.get_dummies(df, columns = ['DefenseAttorney', 'Complainant', 'HearingPlea'], drop_first = True)
        df_model = df_dummied[df_dummied['Contested'] == 1]
        X = df_model.drop(columns = drop_features)
        features = X.columns
        y = df_model['Total_Positive']
        baseline = y.value_counts(normalize = True)
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)
        ss = StandardScaler()
        X_train_sc = ss.fit_transform(X_train)
        X_test_sc = ss.transform(X_test)
        logreg.fit(X_train_sc, y_train)
        cv_train = cross_val_score(logreg, X_train_sc, y_train)
        cv_test = cross_val_score(logreg, X_test_sc, y_test)
    
        coefficients = logreg.coef_
        coef_df = pd.DataFrame(coefficients, columns = features).T
        coef_df['change_odds_ratio'] = coef_df.apply(lambda x: log_odds(x))
        coef_df.rename(columns = {0: 'logreg_coefficient'}, inplace = True)
        coef_df_top = coef_df.sort_values(by = 'logreg_coefficient', ascending = False).head()
        coef_df_bottom = coef_df.sort_values(by = 'logreg_coefficient', ascending = False).tail()
        coef_df_all = pd.concat([coef_df_top, coef_df_bottom])
    
        print("Scores for {} in {}".format(charge, county))
        print("Baseline:", baseline)
        print("Train Accuracy Scores:", cv_train, "Train Average Accuracy:", cv_train.mean())
        print("Test Accuracy Scores:", cv_test, "Test Average Accuracy:", cv_test.mean())
        print("-----")
        print("Factors that Help the Case (Top) and Hurt the Case (Bottom)")
        return coef_df_all
    except:
        pass

In [ ]:
a = interactive(detail_logreg, county = counties, charge = charges)
display(a)

## A/B Testing

1. Is there a statistically significant difference between outcomes based only on race?

Given the simplicity of the model, I want to look more specifically at one charge compared against different court districts in densely populated areas and more rural areas to see if there are statistically significant differences in outcomes based on race. Since speeding infractions are so prevalent, I'll use those for comparison.

In [ ]:
df_full['Race'].value_counts()

In [ ]:
def anova_race(county, charge, *args):
    df_test = df_full[(df_full['ChargeType'] == charge) &
                         (df_full['Court'] == county) &
                         (df_full['Contested'] == 1)]
    if len(args[0]) == 2:
        test = stats.f_oneway(df_test[df_test['Race'] == str(args[0][0])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][1])]['Total_Positive'])
    elif len(args[0]) == 3:
        test = stats.f_oneway(df_test[df_test['Race'] == str(args[0][0])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][1])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][2])]['Total_Positive'])
    elif len(args[0]) == 4:
        test = stats.f_oneway(df_test[df_test['Race'] == str(args[0][0])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][1])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][2])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][3])]['Total_Positive'])
    elif len(args[0]) == 5:
        test = stats.f_oneway(df_test[df_test['Race'] == str(args[0][0])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][1])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][2])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][3])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][4])]['Total_Positive'])
    elif len(args[0]) == 6:
        test = stats.f_oneway(df_test[df_test['Race'] == str(args[0][0])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][1])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][2])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][3])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][4])]['Total_Positive'],
                          df_test[df_test['Race'] == str(args[0][5])]['Total_Positive'])

    percent_race = df_test['Race'].value_counts(normalize = True)
    p_value = test.pvalue
   
    print("Comparing mean outcomes for these populations contesting {} charges in {}:".format(charge, county))
    for i in args[0]:
        print("    ", i)
    print("P-value:", p_value)
    if p_value <= 0.01:
        print("The p-value is sufficiently small that we can reject the null hypothesis and accept the alternative hypothesis: that there is a statistically significant difference in defense outcomes for these groups based on race.")
    if p_value > 0.01:
        print("The p-value is not small enough to reject the null hypothesis. We cannot draw a conclusion about how outcomes differ by race for this charge.")
    print("--------------")
    print("Demographic makeup of defendees contesting {} charges in {}:".format(charge, county))
    print(percent_race)

In [ ]:
# calling function
args = ('White Caucasian(Non-Hispanic)', 'Black(Non-Hispanic)', 'Asian Or Pacific Islander')
anova_race('Fairfax County', 'INF: Speeding', args)

In [ ]:
def anova_defense(county, charge, only_lawyers = True):
    df_test = df_full[(df_full['ChargeType'] == charge) &
                         (df_full['Court'] == county) &
                         (df_full['Contested'] == 1)]
    if only_lawyers == False:
        test = stats.f_oneway(df_test[(df_test['HadLawyer'] == 1) &
                                 (df_test['PublicDefender'] == 0)]['Total_Positive'],
                   df_test[(df_test['HadLawyer'] == 1) &
                          (df_test['PublicDefender'] == 1)]['Total_Positive'],
                   df_test[df_test['HadLawyer'] == 0]['Total_Positive'])
        print("Comparing mean outcomes for defendees with public defenders, private lawyers, and no lawyers contesting {} charges in {} based on defense strategy:".format(charge, county))
    
    elif only_lawyers == True:
        test = stats.f_oneway(df_test[(df_test['HadLawyer'] == 1) &
                                 (df_test['PublicDefender'] == 0)]['Total_Positive'],
                   df_test[(df_test['HadLawyer'] == 1) &
                          (df_test['PublicDefender'] == 1)]['Total_Positive'])
        print("Comparing mean outcomes for defendees with public defenders and private lawyers "
              "contesting {} charges in {} based on defense strategy:".format(charge, county))
    
    percent_had_lawyer = df_test['HadLawyer'].value_counts(normalize = True)
    percent_had_pd = df_test[df_test['HadLawyer'] == 1]['PublicDefender'].value_counts(normalize = True)
    p_value = test.pvalue
    
    print("P-value:", p_value)
    if p_value <= 0.01:
        print("The p-value is sufficiently small that we can reject the null hypothesis and "
        "accept the alternative hypothesis: that there is a statistically significant difference "
        "in defense outcomes for these groups based on defense strategy.")
    if p_value > 0.01:
        print("The p-value is not small enough to reject the null hypothesis. We cannot draw "
        "a conclusion about how outcomes differ by defense strategy for this charge.")
    print("--------------")
    print("Defendees who had lawyers to help contest {} charges in {}:".format(charge, county))
    print(percent_had_lawyer)
    print("Of those with lawyers, the percentage with public defenders:")
    print(percent_had_pd)

In [ ]:
anova_defense('Fairfax County', 'MIS: Drug-related Offenses', only_lawyers = True)

In [ ]:
# INF: Speeding 
# counties around the three main population centers in Virginia
fairfax_speedingINF = df_full[(df_full['ChargeType'] == 'INF: Speeding') &
                                (df_full['Court'] == 'Fairfax County')]

vabeach_speedingINF = df_full[(df_full['ChargeType'] == 'INF: Speeding') &
                                (df_full['Court'] == 'Virginia Beach City')]

henrico_speedingINF = df_full[(df_full['ChargeType'] == 'INF: Speeding') &
                             (df_full['Court'] == 'Henrico County')]

# more rural counties
henry_speedingINF = df_full[(df_full['ChargeType'] == 'INF: Speeding') &
                           (df_full['Court'] == 'Henry County')]

augusta_speedingINF = df_full[(df_full['ChargeType'] == 'INF: Speeding') &
                             (df_full['Court'] == 'Augusta County')]

wythe_speedingINF = df_full[(df_full['ChargeType'] == 'INF: Speeding') &
                             (df_full['Court'] == 'Wythe County')]

In [ ]:
# racial disparity in Fairfax County
stats.f_oneway(fairfax_speedingINF[(fairfax_speedingINF['Race'] == 
                                   'White Caucasian(Non-Hispanic)') &
                                  (fairfax_speedingINF['Contested'] == 1)]['Total_Positive'],
              fairfax_speedingINF[(fairfax_speedingINF['Race'] == 
                                   'Black(Non-Hispanic)') & 
                                 (fairfax_speedingINF['Contested'] == 1)]['Total_Positive'],
              fairfax_speedingINF[(fairfax_speedingINF['Race'] == 
                                   'Hispanic') & 
                                  (fairfax_speedingINF['Contested'] == 1)]['Total_Positive'])

Using a significance level of $\alpha = 0.05$,  the p-value is sufficiently small that we can reject the null hypothesis that the average outcome for different racial groups going to court in Fairfax to fight speeding infraction tickets is the same. 

In [ ]:
# racial disparity in Henry County
stats.f_oneway(henry_speedingINF[(henry_speedingINF['Race'] == 
                                   'White Caucasian(Non-Hispanic)') &
                                  (henry_speedingINF['Contested'] == 1)]['Total_Positive'],
              henry_speedingINF[(henry_speedingINF['Race'] == 
                                   'Black(Non-Hispanic)') & 
                                 (henry_speedingINF['Contested'] == 1)]['Total_Positive'],
              henry_speedingINF[(henry_speedingINF['Race'] == 
                                   'Hispanic') & 
                                  (henry_speedingINF['Contested'] == 1)]['Total_Positive'])

Using a significance level of $\alpha = 0.05$,  the p-value is not small enought to reject the null hypothesis that the average outcome for different racial groups going to court in Henry County to fight speeding infraction tickets is the same. 

In [ ]:
# racial disparity in Henrico County
stats.f_oneway(henrico_speedingINF[(henrico_speedingINF['Race'] == 
                                   'White Caucasian(Non-Hispanic)') &
                                  (henrico_speedingINF['Contested'] == 1)]['Total_Positive'],
              henrico_speedingINF[(henrico_speedingINF['Race'] == 
                                   'Black(Non-Hispanic)') & 
                                 (henrico_speedingINF['Contested'] == 1)]['Total_Positive'],
              henrico_speedingINF[(henrico_speedingINF['Race'] == 
                                   'Hispanic') & 
                                  (henrico_speedingINF['Contested'] == 1)]['Total_Positive'])

Using a significance level of $\alpha = 0.05$,  the p-value is sufficiently small that we can reject the null hypothesis that the average outcome for different racial groups going to court in Henrico County to fight speeding infraction tickets is the same. 

In [ ]:
# racial disparity in Virginia Beach
stats.f_oneway(vabeach_speedingINF[(vabeach_speedingINF['Race'] == 
                                   'White Caucasian(Non-Hispanic)') &
                                  (vabeach_speedingINF['Contested'] == 1)]['Total_Positive'],
              vabeach_speedingINF[(vabeach_speedingINF['Race'] == 
                                   'Black(Non-Hispanic)') & 
                                 (vabeach_speedingINF['Contested'] == 1)]['Total_Positive'],
              vabeach_speedingINF[(vabeach_speedingINF['Race'] == 
                                   'Hispanic') & 
                                  (vabeach_speedingINF['Contested'] == 1)]['Total_Positive'])

Using a significance level of $\alpha = 0.05$,  the p-value is sufficiently small that we can reject the null hypothesis that the average outcome for different racial groups going to court in Virginia Beach to fight speeding infraction tickets is the same. 

In [ ]:
# racial disparity in Augusta County
stats.f_oneway(augusta_speedingINF[(augusta_speedingINF['Race'] == 
                                   'White Caucasian(Non-Hispanic)') &
                                  (augusta_speedingINF['Contested'] == 1)]['Total_Positive'],
              augusta_speedingINF[(augusta_speedingINF['Race'] == 
                                   'Black(Non-Hispanic)') & 
                                 (augusta_speedingINF['Contested'] == 1)]['Total_Positive'],
              augusta_speedingINF[(augusta_speedingINF['Race'] == 
                                   'Hispanic') & 
                                  (augusta_speedingINF['Contested'] == 1)]['Total_Positive'])

Using a significance level of $\alpha = 0.05$,  the p-value is sufficiently small that we can reject the null hypothesis that the average outcome for different racial groups going to court in Augusta County to fight speeding infraction tickets is the same. 

In [ ]:
# racial disparity in Wythe County
stats.f_oneway(wythe_speedingINF[(wythe_speedingINF['Race'] == 
                                   'White Caucasian(Non-Hispanic)') &
                                  (wythe_speedingINF['Contested'] == 1)]['Total_Positive'],
              wythe_speedingINF[(wythe_speedingINF['Race'] == 
                                   'Black(Non-Hispanic)') & 
                                 (wythe_speedingINF['Contested'] == 1)]['Total_Positive'],
              wythe_speedingINF[(wythe_speedingINF['Race'] == 
                                   'Hispanic') & 
                                  (wythe_speedingINF['Contested'] == 1)]['Total_Positive'])

Using a significance level of $\alpha = 0.05$,  the p-value is not small enought to reject the null hypothesis that the average outcome for different racial groups going to court in Wythe County to fight speeding infraction tickets is the same. 

##### Demographic makeup of 6 Counties' Speeding Infraction caseloads

In [ ]:
fairfax_speedingINF['Race'].value_counts(normalize = True)

In [ ]:
vabeach_speedingINF['Race'].value_counts(normalize = True)

In [ ]:
henrico_speedingINF['Race'].value_counts(normalize = True)

In [ ]:
henry_speedingINF['Race'].value_counts(normalize = True)

In [ ]:
augusta_speedingINF['Race'].value_counts(normalize = True)

In [ ]:
wythe_speedingINF['Race'].value_counts(normalize = True)

2. Is there a statistically significant difference in outcomes when a defendant is represented by a private lawyer or public defender?

For this question, I'll look at three different types of charges and compare outcomes with no lawyers, with private lawyers, and with public defenders.

In [ ]:
pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 300)

In [ ]:
# looking for counties that have public defenders for comparison to Fairfax
df_full[(df_full['HadLawyer'] == 1) & (df_full['PublicDefender'] == 1)]['Court'].value_counts().head()

In [ ]:
# Fairfax County, three different charges
fairfax_speedingINF = df_full[(df_full['ChargeType'] == 'INF: Speeding') &
                                (df_full['Court'] == 'Fairfax County') &
                             (df_full['Contested'] == 1)]

fairfax_drugsMIS = df_full[(df_full['ChargeType'] == 'MIS: Drug-related Offenses') &
                                (df_full['Court'] == 'Fairfax County') &
                          (df_full['Contested'] == 1)]

fairfax_recklessMIS = df_full[(df_full['ChargeType'] == 'MIS: Reckless Driving') &
                             (df_full['Court'] == 'Fairfax County') &
                             (df_full['Contested'] == 1)]

# Henry County
newport_speedingINF = df_full[(df_full['ChargeType'] == 'INF: Speeding') &
                                (df_full['Court'] == 'Newport News City') &
                           (df_full['Contested'] == 1)]

newport_drugsMIS = df_full[(df_full['ChargeType'] == 'MIS: Drug-related Offenses') &
                                (df_full['Court'] == 'Newport News City') &
                        (df_full['Contested'] == 1)]

newport_recklessMIS = df_full[(df_full['ChargeType'] == 'MIS: Reckless Driving') &
                             (df_full['Court'] == 'Newport News City') &
                           (df_full['Contested'] == 1)]

##### Speeding Infractions

In [ ]:
# hiring a lawyer for a speeding ticket in Fairfax County
stats.f_oneway(fairfax_speedingINF[(fairfax_speedingINF['HadLawyer'] == 1) & # private lawyer
                                   (fairfax_speedingINF['PublicDefender'] == 0)]['Total_Positive'],
              fairfax_speedingINF[(fairfax_speedingINF['HadLawyer'] == 0) & # self-defense
                                  (fairfax_speedingINF['PublicDefender'] == 0)]['Total_Positive'])

Using a significance level of $\alpha = 0.05$,  the p-value is sufficiently small that we can reject the null hypothesis that the average outcome for people who hire lawyers and don't hire lawyers to fight speeding tickets in Fairfax Virginia is the same.

In [ ]:
# hiring a lawyer for a speeding ticket in Henry County
stats.f_oneway(newport_speedingINF[(newport_speedingINF['HadLawyer'] == 1) & # private lawyer
                                   (newport_speedingINF['PublicDefender'] == 0)]['Total_Positive'],
              newport_speedingINF[(newport_speedingINF['HadLawyer'] == 0) & # self-defense
                                  (newport_speedingINF['PublicDefender'] == 0)]['Total_Positive'])

Using a significance level of $\alpha = 0.05$,  the p-value is sufficiently small that we can reject the null hypothesis that the average outcome for people who hire lawyers and don't hire lawyers to fight speeding tickets in Henry County is the same.

##### Drug-related Offenses

In [ ]:
# hiring a lawyer for a drug-related offense in Fairfax County
stats.f_oneway(fairfax_drugsMIS[(fairfax_drugsMIS['HadLawyer'] == 1) & # private lawyer
                                   (fairfax_drugsMIS['PublicDefender'] == 0)]['Total_Positive'],
              fairfax_drugsMIS[(fairfax_drugsMIS['HadLawyer'] == 0) & # self-defense
                                  (fairfax_drugsMIS['PublicDefender'] == 0)]['Total_Positive'],
              fairfax_drugsMIS[(fairfax_drugsMIS['HadLawyer'] == 1) & # public defender
                                  (fairfax_drugsMIS['PublicDefender'] == 1)]['Total_Positive'])

In [ ]:
# hiring a lawyer/public defender for a drug-related offense in Fairfax
stats.f_oneway(fairfax_drugsMIS[(fairfax_drugsMIS['HadLawyer'] == 1) & # private lawyer
                                   (fairfax_drugsMIS['PublicDefender'] == 0)]['Total_Positive'],
              fairfax_drugsMIS[(fairfax_drugsMIS['HadLawyer'] == 1) & # public defender
                                  (fairfax_drugsMIS['PublicDefender'] == 1)]['Total_Positive'])

The p-value is significantly small enough in the first test to reject the null hypothesis that outcomes are the same whether someone defends themselves against a drug charge, hires a lawyer, or a public defender. Looking at the second test, the p-value is not sufficiently small to reject the null hypothesis (that outcomes for those who hired private lawyers and public defenders is the same). 

In [ ]:
# hiring a lawyer for a drug-related offense in Fairfax County
stats.f_oneway(newport_drugsMIS[(newport_drugsMIS['HadLawyer'] == 1) & # private lawyer
                                   (newport_drugsMIS['PublicDefender'] == 0)]['Total_Positive'],
              newport_drugsMIS[(newport_drugsMIS['HadLawyer'] == 0) & # self-defense
                                  (newport_drugsMIS['PublicDefender'] == 0)]['Total_Positive'],
              newport_drugsMIS[(newport_drugsMIS['HadLawyer'] == 1) & # public defender
                                  (newport_drugsMIS['PublicDefender'] == 1)]['Total_Positive'])

In [ ]:
# hiring a lawyer for a drug-related offense in Fairfax County
stats.f_oneway(newport_drugsMIS[(newport_drugsMIS['HadLawyer'] == 1) & # private lawyer
                                   (newport_drugsMIS['PublicDefender'] == 0)]['Total_Positive'],
              newport_drugsMIS[(newport_drugsMIS['HadLawyer'] == 1) & # public defender
                                  (newport_drugsMIS['PublicDefender'] == 1)]['Total_Positive'])

Looking at the first test, the p-value is sufficiently small to reject the null hypothesis that there is a difference in outcomes for defendants fighting drug charges in Newport News based on whether they hired lawyers, defended themselves, or had public defenders. So outcomes are probably different; in the second test, the p-value is still sufficiently small for us to reject the null that there is no difference between hiring a private lawyer and public defender.

##### Reckless Driving

In [ ]:
# hiring a lawyer for reckless driving in Fairfax County
stats.f_oneway(fairfax_recklessMIS[(fairfax_recklessMIS['HadLawyer'] == 1) & # private lawyer
                                   (fairfax_recklessMIS['PublicDefender'] == 0)]['Total_Positive'],
              fairfax_recklessMIS[(fairfax_recklessMIS['HadLawyer'] == 0) & # self-defense
                                  (fairfax_recklessMIS['PublicDefender'] == 0)]['Total_Positive'],
              fairfax_recklessMIS[(fairfax_recklessMIS['HadLawyer'] == 1) & # public defender
                                  (fairfax_recklessMIS['PublicDefender'] == 1)]['Total_Positive'])

In [ ]:
# hiring a lawyer for reckless driving in Fairfax County
stats.f_oneway(fairfax_recklessMIS[(fairfax_recklessMIS['HadLawyer'] == 1) & # private lawyer
                                   (fairfax_recklessMIS['PublicDefender'] == 0)]['Total_Positive'],
              fairfax_recklessMIS[(fairfax_recklessMIS['HadLawyer'] == 1) & # public defender
                                  (fairfax_recklessMIS['PublicDefender'] == 1)]['Total_Positive'])

The p-value is significantly small to reject the null hypothesis and accept the alternative (that there is a statistically significant difference in outcome for reckless driving cases in Fairfax County based on whether someone hires a lawyer, defends themselves, or hires a public defender.)

In [ ]:
# hiring a lawyer for a drug-related offense in Fairfax County
stats.f_oneway(newport_recklessMIS[(newport_recklessMIS['HadLawyer'] == 1) & # private lawyer
                                   (newport_recklessMIS['PublicDefender'] == 0)]['Total_Positive'],
              newport_recklessMIS[(newport_recklessMIS['HadLawyer'] == 0) & # self-defense
                                  (newport_recklessMIS['PublicDefender'] == 0)]['Total_Positive'],
              newport_recklessMIS[(newport_recklessMIS['HadLawyer'] == 1) & # public defender
                                  (newport_recklessMIS['PublicDefender'] == 1)]['Total_Positive'])

The p-value is not sufficiently small to reject the null hypothesis that the average outcome for reckless driving defendants is the same (whether they hire private lawyers, public defenders, or defend themselves.)